# 🗺️ Bengaluru Travel Planner - Multi-Agent System

An AI-powered travel itinerary generator using Google ADK with specialized sub-agents:
- **ResearchAgent** - Attraction details, timings, prices
- **EventsAgent** - Events happening on your travel date
- **TransportAgent** - Distance, travel time, fare estimates
- **FoodAgent** - Restaurant recommendations
- **WeatherAgent** - Weather forecast and advice
- **ItineraryCoordinator** - Orchestrates all agents to create the perfect plan

In [1]:
print("Jai Shree Shyam!")

Jai Shree Shyam!


## 1. Setup & Configuration

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load API key from environment variable - NEVER hardcode API keys!
# Get your key from: https://console.cloud.google.com/apis/credentials
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Please set GOOGLE_API_KEY in your .env file")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ API Key configured")

In [3]:
# Import the multi-agent system
from google.adk.runners import InMemoryRunner
from travel_planner.travel_agents import (
    itinerary_coordinator,
    research_agent,
    events_agent,
    transport_agent,
    food_agent,
    weather_agent,
    get_all_agents,
)

print("✅ All agents imported successfully!")
print(f"\nAvailable agents: {list(get_all_agents().keys())}")

✅ All agents imported successfully!

Available agents: ['coordinator', 'research', 'events', 'transport', 'food', 'weather']


## 2. Initialize the Runner

In [4]:
# Create the runner with the main coordinator
runner = InMemoryRunner(agent=itinerary_coordinator)
print("✅ Runner initialized with ItineraryCoordinator")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


✅ Runner initialized with ItineraryCoordinator


## 3. Test Individual Tools (Optional)

Let's verify the custom tools work correctly:

In [5]:
# Test Distance Matrix
from travel_planner.tools import get_distance_matrix, geocode_location, find_nearby_places

# Test geocoding
geo_result = await geocode_location("NIFT College, HSR Layout, Bengaluru")
print("📍 Geocoding Result:")
print(geo_result)
print(f"   Address: {geo_result.get('formatted_address')}")
print(f"   Coordinates: {geo_result.get('latitude')}, {geo_result.get('longitude')}")

📍 Geocoding Result:
{'query': 'NIFT College, HSR Layout, Bengaluru', 'formatted_address': 'NIFT College, 27th Main Rd, PWD Quarters, 1st Sector, HSR Layout, Bengaluru, Karnataka 560102, India', 'latitude': 12.912156, 'longitude': 77.651983, 'place_id': 'ChIJD2mGPp0UrjsRfI_OWXpqXzM', 'location_type': 'GEOMETRIC_CENTER', 'address_components': {'area': 'HSR Layout', 'city': 'Bengaluru', 'state': 'Karnataka', 'pincode': '560102'}, 'google_maps_url': 'https://www.google.com/maps?q=12.912156,77.651983', 'status': 'OK'}
   Address: NIFT College, 27th Main Rd, PWD Quarters, 1st Sector, HSR Layout, Bengaluru, Karnataka 560102, India
   Coordinates: 12.912156, 77.651983


In [6]:
# Test distance calculation
distance_result = await get_distance_matrix(
    origins="The Den, Bengaluru",
    destinations="NIFT College, HSR Layout, Bengaluru",
    mode="driving",
    departure_time="now"
)
print(distance_result)
print("🚗 Distance Matrix Result:")
print(f"   Distance: {distance_result.get('distance')}")
print(f"   Duration: {distance_result.get('duration')}")
if 'duration_in_traffic' in distance_result:
    print(f"   With Traffic: {distance_result.get('duration_in_traffic')}")

{'origin_query': 'The Den, Bengaluru', 'destination_query': 'NIFT College, HSR Layout, Bengaluru', 'origin_resolved': 'ITPL Main Rd, KIADB Export Promotion Industrial Area, Whitefield, Bengaluru, Karnataka 560066, India', 'destination_resolved': 'NIFT College, 27th Main Rd, PWD Quarters, 1st Sector, HSR Layout, Bengaluru, Karnataka 560102, India', 'origin_coordinates': '12.9876819,77.7333398', 'destination_coordinates': '12.912156,77.651983', 'distance': '17.3 km', 'distance_meters': 17313, 'distance_km': 17.31, 'duration': '38 mins', 'duration_seconds': 2288, 'duration_minutes': 38, 'travel_mode': 'driving', 'status': 'OK', 'duration_in_traffic': '43 mins', 'duration_in_traffic_seconds': 2595, 'duration_in_traffic_minutes': 43}
🚗 Distance Matrix Result:
   Distance: 17.3 km
   Duration: 38 mins
   With Traffic: 43 mins


In [7]:
# Test nearby places
places_result = await find_nearby_places(
    location="HSR Layout, Bengaluru",
    place_type="restaurant",
    radius_meters=2000
)
print(f"🍽️ Found {places_result.get('places_found')} restaurants near HSR Layout:")
for place in places_result.get('places', [])[:10]:
    print(f"   - {place['name']} ({place.get('rating', 'N/A')}⭐)")

🍽️ Found 10 restaurants near HSR Layout:
   - The Dome Cafe - HSR Layout (4.2⭐)
   - Copper Kitchen Restaurant (4.4⭐)
   - Go Native HSR (4.4⭐)
   - Olive Street Food Cafe - HSR LAYOUT (4.6⭐)
   - Roxie (4.4⭐)
   - Mughal Treat (4.1⭐)
   - SOMRAS BAR & KITCHEN (4.8⭐)
   - Light of Italy | PURE VEG (4.7⭐)
   - Andhra Vilas (4.4⭐)
   - Dhaba Stories HSR (4.1⭐)


## 4. Generate Full Day Itinerary

### Query Format
```
type: fullday/morning/evening/night (Required)
people: friends/family/couple/corporate/solo (Required)
numberOfPeople: Number (Required)
location: Starting address (Required)
dateOfPlan: Date (Required)
startTime: Time (Required)
occasion: Why the trip (Optional)
inclusions: Activities like temple, zoo, nature, food, shopping (Optional)
budget: Total amount in INR (Optional)
transportMode: auto/cab/metro/mixed (Optional)
remarks: Special requests (Optional)
```

In [8]:
# Example 1: Full day trip with friends
query1 = """
type: fullday,
people: friends,
numberOfPeople: 8,
occasion: weekend fun,
inclusions: temple, zoo, nature, good food,
location: NIFT College, HSR Layout, Bengaluru,
dateOfPlan: 5 December 2025,
startTime: 10 AM
"""

print("🚀 Generating itinerary...")
print("This may take 30-60 seconds as multiple agents coordinate...\n")

response1 = await runner.run_debug(query1)

🚀 Generating itinerary...
This may take 30-60 seconds as multiple agents coordinate...


 ### Created new session: debug_session_id

User > 
type: fullday,
people: friends,
numberOfPeople: 8,
occasion: weekend fun,
inclusions: temple, zoo, nature, good food,
location: NIFT College, HSR Layout, Bengaluru,
dateOfPlan: 5 December 2025,
startTime: 10 AM



App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


ItineraryCoordinator > Okay, I will create a fun-filled day itinerary for you and your friends in Bengaluru on December 5, 2025, starting at 10 AM from NIFT College, HSR Layout. The plan includes a temple visit, zoo, nature, and good food.

First, let's check the weather forecast for the day.



App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


ItineraryCoordinator > Okay, the weather forecast indicates a 35% chance of rain during the day. I will prioritize the outdoor activities in the morning and keep indoor alternatives ready.

Next, I'll check for any events happening in Bengaluru on December 5, 2025.



...

ItineraryCoordinator > SLV Ragigudda is a good option, being close to the temple. Now let's find some restaurants near Bannerghatta National Park.



App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


ItineraryCoordinator > Hotel Green Way is the closest, at 1.19 km from the park and seems suitable for the group.

Now that I have all the information, I can create the itinerary.



App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


ItineraryCoordinator > Okay, I am ready to present the final itinerary.

## 🗓️ Fullday Itinerary for Bengaluru
**Date:** December 5, 2025 | **Group:** 8 Friends | **Occasion:** Weekend Fun

### 🌤️ Weather
Light rain during the day, partly cloudy at night. High of 25°C (77°F), Low of 18°C (65°F). Wear light layers and carry an umbrella. Plan outdoor activities before 3 PM.

### 🎉 Events Today
Indian Cacao & Craft Chocolate Festival (3:00 PM - 7:30 PM) at Sabha Blr, Kamaraj Rd, Shivaji Nagar (Entry INR 350).

---

### 📍 Detailed Itinerary

**10:00 AM - Visit Ragigudda Sri Prasanna Anjaneyaswamy Temple**
📍 Location: Ragigudda, 9th Block, Jayanagar, Bengaluru, Karnataka 560069
⏱️ Duration: 1.5 hours
💰 Cost: Free
ℹ️ A beautiful temple dedicated to Lord Hanuman. Be mindful of the dress code (modest clothing).

🚗 **Travel to Bannerghatta National Park**
- Distance: 14 km
- Duration: 45 mins (with traffic: 60 mins)
- Mode: Tempo Traveler (recommended for 8 people)
- Fare: ₹1500 (approx)

**12:

In [16]:
# Example 2: Evening plan for couple
query2 = """
type: evening,
people: couple,
numberOfPeople: 2,
occasion: anniversary dinner,
inclusions: romantic restaurant, sunset view, dessert,
budget: 5000,
budgetMode: flexible,
location: Indiranagar, Bengaluru,
dateOfPlan: 14 February 2026,
startTime: 4 PM,
remarks: prefer rooftop or outdoor seating
"""

response2 = await runner.run_debug(query2)


 ### Continue session: debug_session_id

User > 
type: evening,
people: couple,
numberOfPeople: 2,
occasion: anniversary dinner,
inclusions: romantic restaurant, sunset view, dessert,
budget: 5000,
budgetMode: flexible,
location: Indiranagar, Bengaluru,
dateOfPlan: 14 February 2026,
startTime: 4 PM,
remarks: prefer rooftop or outdoor seating



App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agent

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Tool use with function calling is unsupported', 'status': 'INVALID_ARGUMENT'}}

In [10]:
# Example 3: Family day out with kids
query3 = """
type: fullday,
people: family,
numberOfPeople: 5,
occasion: kids holiday,
inclusions: amusement park, science museum, ice cream,
budget: 8000,
transportMode: cab,
location: Whitefield, Bengaluru,
dateOfPlan: 25 December 2025,
startTime: 9 AM,
remarks: 2 kids aged 8 and 12, need kid-friendly activities
"""

response3 = await runner.run_debug(query3)

NameError: name 'runner' is not defined

## 5. Using the Structured Query Helper

In [8]:
from travel_planner.runner import TravelQuery, TravelPlannerRunner

# Create a structured query
my_trip = TravelQuery(
    type="fullday",
    people="corporate",
    number_of_people=15,
    location="Koramangala, Bengaluru",
    date_of_plan="20 December 2025",
    start_time="11 AM",
    occasion="team outing",
    inclusions=["adventure", "team activities", "lunch", "dinner"],
    budget=30000,
    budget_mode="strict",
    remarks="Need activities that encourage team bonding"
)

print(f"Generated query: {my_trip.to_query_string()}")

Generated query: type:fullday, people:corporate, numberOfPeople:15, location:Koramangala, Bengaluru, dateOfPlan:20 December 2025, startTime:11 AM, occasion:team outing, inclusions:adventure, team activities, lunch, dinner, budget:30000, budgetMode:strict, remarks:Need activities that encourage team bonding


In [9]:
# Run with the structured query
planner = TravelPlannerRunner(debug=True)
result = await planner.plan_trip(my_trip)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".



TRAVEL PLANNER - Debug Mode
Query: type:fullday, people:corporate, numberOfPeople:15, location:Koramangala, Bengaluru, dateOfPlan:20 December 2025, startTime:11 AM, occasion:team outing, inclusions:adventure, team activities, lunch, dinner, budget:30000, budgetMode:strict, remarks:Need activities that encourage team bonding


 ### Created new session: debug_session_id

User > type:fullday, people:corporate, numberOfPeople:15, location:Koramangala, Bengaluru, dateOfPlan:20 December 2025, startTime:11 AM, occasion:team outing, inclusions:adventure, team activities, lunch, dinner, budget:30000, budgetMode:strict, remarks:Need activities that encourage team bonding


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


## 6. Test Individual Agents

You can also test each sub-agent individually:

In [4]:
# Test Events Agent
events_runner = InMemoryRunner(agent=events_agent)
events_response = await events_runner.run_debug(
    "Find events happening in Bengaluru on 5 December 2025, suitable for a group of friends"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Find events happening in Bengaluru on 5 December 2025, suitable for a group of friends
EventsAgent > Here are some events happening in Bengaluru on December 5, 2025, that would be suitable for a group of friends:

**Comedy Shows:**

*   **Event Name:** Allow Me!: A Standup Comedy Show by Rahul Dua
*   **Venue/Location:** MLR Convention Centre: JP Nagar, Bangalore
*   **Date and Time:** Friday, December 5, 2025, at 8:00 PM IST
*   **Category:** Comedy
*   **Ticket Price:** Starting at INR 799
*   **Booking Link:** Check specific ticketing platforms like BookMyShow or Insider.in for booking.
*   **Relevance to user's plan:** A stand-up comedy show is a great way for a group of friends to have a fun and entertaining evening. Rahul Dua is a popular comedian, ensuring a good dose of laughter.

**Festivals & Fairs:**

*   **Event Name:** Indian Cacao & Craft Chocolate Festival
*   **Venue/Location:** Sabha Blr, Kamaraj Rd, Shivaji Nagar, Be

In [ ]:
# Test Food Agent
food_runner = InMemoryRunner(agent=food_agent)
food_response = await food_runner.run_debug(
    "Find restaurants near Lalbagh Botanical Garden for a group of 8 friends, budget around Rs 500 per person"
)

In [ ]:
# Test Transport Agent
transport_runner = InMemoryRunner(agent=transport_agent)
transport_response = await transport_runner.run_debug(
    "Calculate travel time and fare from HSR Layout to Bannerghatta Zoo for 8 people on a Saturday morning"
)

---

## Next Steps: Masumi Integration

To deploy this agent on Masumi Network:
1. Wrap with FastAPI (MIP-003 compliant endpoints)
2. Add payment verification
3. Register agent on Cardano blockchain

See `api/main.py` for the API implementation.